In [1]:
# imports
import operator
import random
import numpy

from deap import base, benchmarks, creator, tools

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_boston
from sklearn import metrics

import matplotlib.pyplot as plt

In [2]:
db = load_boston()

In [3]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, smin=None, smax=None, best=None)

In [4]:
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

In [5]:
def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))

In [6]:
toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

In [7]:
def main():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best

In [8]:
if __name__ == "__main__":
    results = main()
    print(results[1])

gen	evals	avg      	std      	min      	max      
0  	5    	0.0579746	0.0213108	0.0182592	0.0804723
1  	5    	0.0640956	0.0403429	0.00290995	0.125144 
2  	5    	0.0728735	0.0422149	0.0277195 	0.149671 
3  	5    	0.128381 	0.0421375	0.0858898 	0.206349 
4  	5    	0.127832 	0.0687398	0.0330103 	0.247878 
5  	5    	0.165605 	0.0470096	0.0868867 	0.223278 
6  	5    	0.289639 	0.141841 	0.16538   	0.5651   
7  	5    	0.315355 	0.0905684	0.19075   	0.441522 
8  	5    	0.32113  	0.224844 	0.06965   	0.684372 
9  	5    	0.317613 	0.378348 	0.0163142 	1.05538  
10 	5    	0.215161 	0.124993 	0.104742  	0.458114 
11 	5    	0.467923 	0.470577 	0.0370451 	1.36939  
12 	5    	0.362906 	0.358396 	0.0160834 	0.998118 
13 	5    	0.303863 	0.253942 	0.0103434 	0.692772 
14 	5    	0.492644 	0.394268 	0.0575112 	1.23229  
15 	5    	0.273311 	0.154317 	0.00736206	0.451122 
16 	5    	0.448229 	0.37153  	0.0854094 	1.16346  
17 	5    	0.292527 	0.223255 	0.00554358	0.542638 
18 	5    	0.712439 	0.634543 	0.0

644	5    	0.343549 	0.163761 	0.156686  	0.607382 
645	5    	0.332638 	0.145678 	0.0498998 	0.45412  
646	5    	0.462753 	0.307617 	0.251076  	1.07023  
647	5    	0.382368 	0.0842635	0.30506   	0.526273 
648	5    	0.443577 	0.458917 	0.0536107 	1.32392  
649	5    	0.475251 	0.0891348	0.356421  	0.587777 
650	5    	0.326905 	0.272173 	0.0523265 	0.8233   
651	5    	0.434844 	0.336821 	0.12752   	1.06254  
652	5    	0.34183  	0.178456 	0.0438041 	0.531581 
653	5    	0.561304 	0.382941 	0.0882807 	1.14706  
654	5    	0.485694 	0.398835 	0.214432  	1.27489  
655	5    	0.454449 	0.203665 	0.277615  	0.851985 
656	5    	0.555353 	0.307958 	0.326444  	1.15212  
657	5    	0.5059   	0.346365 	0.153615  	1.12557  
658	5    	0.633775 	0.554344 	0.295479  	1.73425  
659	5    	0.534557 	0.168102 	0.38318   	0.814151 
660	5    	0.980571 	0.56605  	0.223635  	1.73061  
661	5    	0.6058   	0.151099 	0.467941  	0.866137 
662	5    	0.68492  	0.447242 	0.308351  	1.3151   
663	5    	0.585171 	0.243659 	0